In [6]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np

In [7]:
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

In [19]:
data = yf.download('TCS', start=start_date,
                 end=end_date,
                 progress=False)
data["Date"] = data.index
data = data[[ "Date","Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-11-01,35.000000,37.000000,32.099998,36.200001,33.839130,14667600
1,2013-11-04,36.799999,36.799999,34.689999,35.349998,33.044563,1586600
2,2013-11-05,35.110001,35.500000,34.820000,35.349998,33.044563,337700
3,2013-11-06,35.610001,36.490002,35.610001,35.900002,33.558697,549600
4,2013-11-07,36.799999,36.799999,34.119999,35.310001,33.007175,883700
...,...,...,...,...,...,...,...
2442,2023-07-19,3.300000,3.520000,3.280000,3.490000,3.490000,508900
2443,2023-07-20,3.460000,3.460000,3.200000,3.210000,3.210000,284500
2444,2023-07-21,3.230000,3.300000,3.200000,3.220000,3.220000,282100
2445,2023-07-24,3.270000,3.360000,3.210000,3.330000,3.330000,355400


In [9]:
fig = go.Figure(data=[go.Candlestick(x=data["Date"],
                                  open=data["Open"],
                                  high=data["High"],
                                  low=data["Low"],
                                  close=data["Close"])])
fig.update_layout(title = "TCS Stock Price Analysis",
                  xaxis_rangeslider_visible=True)
fig.show()

In [10]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Low          0.999485
High         0.999461
Adj Close    0.999324
Open         0.998762
Volume       0.120364
Name: Close, dtype: float64


/var/folders/m0/clg07x494d9c923nvm2j8p940000gn/T/ipykernel_13038/2923706208.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



#### Training LSTM Model

In [11]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2, random_state=42)

In [12]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape = (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss= 'mean_squared_error')
model.fit(xtrain, ytrain,batch_size=1, epochs=30)

Epoch 1/30
1957/1957 [==============================] - 3s 1ms/step - loss: 5.1680
Epoch 2/30
1957/1957 [==============================] - 2s 1ms/step - loss: 1.0826
Epoch 3/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.8767
Epoch 4/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.4578
Epoch 5/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.7168
Epoch 6/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.5751
Epoch 7/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.3657
Epoch 8/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.5074
Epoch 9/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.3869
Epoch 10/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.4076
Epoch 11/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.2997
Epoch 12/30
1957/1957 [==============================] - 2s 1ms/step - loss: 0.2827
E

In [14]:
features = np.array([[177, 180, 177, 74916600]])
model.predict(features)

1/1 [==============================] - 0s 378ms/step


array([[50.0258]], dtype=float32)